## 1. 准备数据

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    x = x.reshape(x.shape[0], 28, 28,1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28,1)
    
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    # Take 20000 samples and randomly shuffle them into multiple batches of size 32
    ds = ds.take(20000).shuffle(20000).batch(32)
    
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_ds = test_ds.map(prepare_mnist_features_and_labels)
    # Take 20000 samples and randomly shuffle them into batches of size 20000
    test_ds = test_ds.take(20000).shuffle(20000).batch(20000)
    return ds, test_ds

# Normalize data
def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

## 2. 建立模型

In [7]:
class myConvModel(keras.Model):
    def __init__(self):
        super(myConvModel, self).__init__()
        # 32 filters, size 5x5, ReLU activation function, and 'same' padding
        self.l1_conv = Conv2D(filters=32,
                              kernel_size=(5, 5),
                              activation='relu',
                              padding='same')
        # 64 filters, size 5x5, ReLU activation function, and 'same' padding
        self.l2_conv = Conv2D(filters=64,
                              kernel_size=(5, 5),
                              activation='relu',
                              padding='same')
        # pool size 2x2, stride 2
        self.pool = MaxPooling2D(pool_size=(2, 2), strides=2)
        
        self.flat = Flatten()
        self.dense1 = layers.Dense(100, activation='tanh')
        self.dense2 = layers.Dense(10)

    def call(self, x):
        h1 = self.l1_conv(x) 
        h1_pool = self.pool(h1) 
        h2 = self.l2_conv(h1_pool)
        h2_pool = self.pool(h2) 
        flat_h = self.flat(h2_pool)
        dense1 = self.dense1(flat_h)
        logits = self.dense2(dense1)
        probs = tf.nn.softmax(logits, axis=-1)
        return probs
    
model = myConvModel()
optimizer = optimizers.Adam(0.001)

## 3. 定义loss以及train loop

In [3]:
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

def train_one_step(model, optimizer, x, y):

    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    grads = tape.gradient(loss, model.trainable_variables)
    # update to weights
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

def test_step(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

def train(epoch, model, optimizer, ds):
    loss = 0.0
    accuracy = 0.0
    for step, (x, y) in enumerate(ds):
        loss, accuracy = train_one_step(model, optimizer, x, y)

        if step % 500 == 0:
            print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

    return loss, accuracy
def test(model, ds):
    loss = 0.0
    accuracy = 0.0
    for step, (x, y) in enumerate(ds):
        loss, accuracy = test_step(model, x, y)

        
    print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

    return loss, accuracy

In [4]:
train_ds, test_ds = mnist_dataset()
for epoch in range(2):
    loss, accuracy = train(epoch, model, optimizer, train_ds)
loss, accuracy = test(model, test_ds)

epoch 0 : loss 2.30187 ; accuracy 0.15625
epoch 0 : loss 1.4654369 ; accuracy 1.0
epoch 1 : loss 1.5232987 ; accuracy 0.9375
epoch 1 : loss 1.5188589 ; accuracy 0.9375
test loss 1.4793775 ; accuracy 0.984


In [5]:
model2 = myConvModel()
model2.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
train_ds, test_ds = mnist_dataset()
model2.fit(train_ds, epochs=5)
model2.evaluate(test_ds)

Epoch 1/5
625/625 [==============================] - 21s 33ms/step - loss: 0.1554 - accuracy: 0.9524
Epoch 2/5
625/625 [==============================] - 19s 31ms/step - loss: 0.0412 - accuracy: 0.9876
Epoch 3/5
625/625 [==============================] - 20s 32ms/step - loss: 0.0231 - accuracy: 0.9936
Epoch 4/5
625/625 [==============================] - 20s 32ms/step - loss: 0.0119 - accuracy: 0.9965
Epoch 5/5
1/1 [==============================] - 3s 3s/step - loss: 0.0342 - accuracy: 0.9886


[0.03416329249739647, 0.9886]

In [6]:
model3 = keras.Sequential([
    Conv2D(32, (5, 5), activation='relu', padding='same'),
    MaxPooling2D(pool_size=2, strides=2),
    Conv2D(64, (5, 5), activation='relu', padding='same'),
    MaxPooling2D(pool_size=2, strides=2),
    Flatten(), #N*7*7*64 =>N*3136
    layers.Dense(128, activation='tanh'), #N*128
    layers.Dense(10, activation='softmax')]) #N*10
optimizer = optimizers.Adam(0.0001)

model3.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
train_ds, test_ds = mnist_dataset()
model3.fit(train_ds, epochs=5)
model3.evaluate(test_ds)

Epoch 1/5
625/625 [==============================] - 20s 32ms/step - loss: 0.5128 - accuracy: 0.8683
Epoch 2/5
625/625 [==============================] - 19s 31ms/step - loss: 0.1456 - accuracy: 0.9595
Epoch 3/5
625/625 [==============================] - 19s 30ms/step - loss: 0.0966 - accuracy: 0.9718
Epoch 4/5
625/625 [==============================] - 19s 30ms/step - loss: 0.0747 - accuracy: 0.9776
Epoch 5/5
1/1 [==============================] - 3s 3s/step - loss: 0.0576 - accuracy: 0.9821


[0.05759537220001221, 0.9821]